In [1]:
import pandas as pd

df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))
df = df.reset_index(drop=True)

df=df[df.Borough != 'Not assigned']

df=df.groupby(['Postcode', 'Borough']).Neighbourhood.apply(', '.join).reset_index()

df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']

df1 = pd.read_csv('https://cocl.us/Geospatial_data')

df_new = pd.concat([df, df1], axis=1)

df_new.drop(['Postal Code'], axis=1)

df_new.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [23]:
import numpy as np

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import json
from pandas.io.json import json_normalize

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


Use geopy library to get the latitude and longitude values of Toronto

In [5]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top

In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

simplify the above map and segment and cluster only the neighborhoods in Scarborough

In [11]:
scarborough_data = df_new[df_new['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


get the geographical coordinates of Scarborough

In [12]:
address = 'Scarborough, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


visualize Scarborough neighborhoods

In [13]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'GKUAF244OPVYY01Z33OBZRVVQ5XFFLMKSKR5PNF425QFG0CU' 
CLIENT_SECRET = 'CENIDR0OMZIIKAPI0CT145ZYU3J2OQUKFLRMRD4BLJSJRQZI' 
VERSION = '20180605' 

explore a neighborhood, get the neighborhood's name

In [18]:
scarborough_data.loc[3, 'Neighbourhood']

'Woburn'

Get the neighborhood's latitude and longitude values

In [20]:
neighborhood_latitude = scarborough_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[3, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


get the top 100 venues within a radius of 500 meters

In [25]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GKUAF244OPVYY01Z33OBZRVVQ5XFFLMKSKR5PNF425QFG0CU&client_secret=CENIDR0OMZIIKAPI0CT145ZYU3J2OQUKFLRMRD4BLJSJRQZI&v=20180605&ll=43.7709921,-79.21691740000001&radius=500&limit=100'

Send the GET request and examine the results

In [28]:
import requests 
results = requests.get(url).json()

borrow the get_category_type function

In [29]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean the json and structure it into a pandas dataframe.

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()



,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.770037,-79.221156
1,Tim Hortons,Coffee Shop,43.770827,-79.223078
2,Korean Grill House,Korean Restaurant,43.770812,-79.214502
3,Leslie & Giles Insurance Limited,Insurance Office,43.773116,-79.219559


how many venues were returned by Foursquare?

In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


create a function to repeat the same process to all the neighborhoods in Scarborough

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

run the above function on each neighborhood and create a new dataframe

In [36]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


check the size of the resulting dataframe

In [37]:
print(scarborough_venues.shape)
scarborough_venues.head()

(85, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


check how many venues were returned for each neighborhood

In [39]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


find out how many unique categories can be curated from all the returned venues

In [40]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 52 uniques categories.


Analyze Each Neighborhood

In [41]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [42]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.0,0.000000,0.00,...,0.0,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.25,...,0.0,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.142857,0.142857,0.142857,0.0,0.000000,0.0,0.000000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.142857,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.100000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.1,0.000000,0.000000,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.0,0.100000,0.000000,0.00,0.00,0.0,0.000000,0.100000,0.000000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.142857,0.142857
8,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.166667,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000
9,"Guildwood, Morningside, West Hill",0.0,0.000000,0.000000,0.000000,0.0,0.142857,0.0,0.000000,0.00,...,0.0,0.142857,0.000000,0.00,0.00,0.0,0.142857,0.000000,0.000000,0.000000


print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0               Lounge  0.25
1       Breakfast Spot  0.25
2         Skating Rink  0.25
3       Clothing Store  0.25
4  American Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0                 Park  0.33
1           Playground  0.33
2          Coffee Shop  0.33
3  American Restaurant  0.00
4             Pharmacy  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2                   Café  0.25
3  General Entertainment  0.25
4               Pharmacy  0.00


----Cedarbrae----
                  venue  freq
0                Bakery  0.14
1                  Bank  0.14
2       Thai Restaurant  0.14
3    Athletics & Sports  0.14
4  Caribbean Restaurant  0.14


----Clairlea, Golden Mile, Oakridge----
           venue  freq
0         Bakery   0.2
1       Bus Line   0.2
2   Intersection   0.1
3  Metro 

write a function to sort the venues in descending order

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the new dataframe and display the top 10 venues for each neighborhood.

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Coffee Shop,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Park,Soccer Field,Intersection,Bus Station,Fast Food Restaurant,Metro Station,College Stadium,Grocery Store


Run k-means to cluster the neighborhood into 5 clusters

In [47]:
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 1, 1, 1, 3, 1, 1, 1], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [63]:
neighborhoods_venues_sorted.insert(0, 'label1', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarborough_merged.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,label1,labels,label,Label,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,1.0,1.0,1.0,1.0,...,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497,0.0,0.0,0.0,0.0,...,History Museum,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,1.0,1.0,1.0,1.0,...,Spa,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,2.0,2.0,2.0,2.0,...,Coffee Shop,Korean Restaurant,Insurance Office,Bakery,Convenience Store,Hakka Restaurant,Athletics & Sports,Grocery Store,General Entertainment,Fried Chicken Joint
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,1.0,1.0,1.0,1.0,...,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store


visualize the resulting clusters

In [67]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['label1']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

examine each cluster

In [68]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,label1,labels,label,Label,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,-79.160497,0.0,0.0,0.0,0.0,0.0,0.0,History Museum,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [69]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,label1,labels,label,Label,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,1.0,1.0,1.0,1.0,1.0,1.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
2,Scarborough,-79.188711,1.0,1.0,1.0,1.0,1.0,1.0,Spa,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment
4,Scarborough,-79.239476,1.0,1.0,1.0,1.0,1.0,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Convenience Store,Grocery Store
6,Scarborough,-79.262029,1.0,1.0,1.0,1.0,1.0,1.0,Discount Store,Coffee Shop,Bus Station,Department Store,Convenience Store,College Stadium,History Museum,Hakka Restaurant,Grocery Store,General Entertainment
7,Scarborough,-79.284577,1.0,1.0,1.0,1.0,1.0,1.0,Bakery,Bus Line,Park,Soccer Field,Intersection,Bus Station,Fast Food Restaurant,Metro Station,College Stadium,Grocery Store
9,Scarborough,-79.264848,1.0,1.0,1.0,1.0,1.0,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,-79.273304,1.0,1.0,1.0,1.0,1.0,1.0,Indian Restaurant,Chinese Restaurant,Thrift / Vintage Store,Latin American Restaurant,Pet Store,Vietnamese Restaurant,Breakfast Spot,Bus Line,Athletics & Sports,Hakka Restaurant
11,Scarborough,-79.295849,1.0,1.0,1.0,1.0,1.0,1.0,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
12,Scarborough,-79.262029,1.0,1.0,1.0,1.0,1.0,1.0,Clothing Store,Skating Rink,Breakfast Spot,Lounge,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,-79.304302,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Rental Car Location,Italian Restaurant,Noodle House,Pharmacy,Chinese Restaurant,Fast Food Restaurant,Fried Chicken Joint,Thai Restaurant,Electronics Store


In [70]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,label1,labels,label,Label,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,-79.216917,2.0,2.0,2.0,2.0,2.0,2.0,Coffee Shop,Korean Restaurant,Insurance Office,Bakery,Convenience Store,Hakka Restaurant,Athletics & Sports,Grocery Store,General Entertainment,Fried Chicken Joint
14,Scarborough,-79.284577,2.0,2.0,2.0,2.0,2.0,2.0,Park,Playground,Coffee Shop,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [71]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,label1,labels,label,Label,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,-79.239476,3.0,3.0,3.0,3.0,3.0,3.0,American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [72]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Longitude,label1,labels,label,Label,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,-79.239476,4.0,4.0,4.0,4.0,4.0,4.0,Grocery Store,Playground,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
